In [88]:
import os
import json
import numpy as np
import pandas as pd

In [89]:
CSV_PATH = "dataset/final/dataprep_pead_event_2010_2019.csv"
OUT_DIR  = "pead_preproc"
os.makedirs(OUT_DIR, exist_ok=True)

In [90]:
FEATS = [

    # ===== Fundamental – níveis MET =====
    "RL_MET", "LL_MET", "EBITDA_MET",
    "Preco_Abertura_MET", "Preco_Fechamento_MET",
    "LPA_MET", "ROA_MET", "ROE_MET", "MEB_MET",
    "CRESC_RL_12M_MET", "CRESC_LL_12M_MET", "CRESC_EBITDA_12M_MET",
    "CAPEX_MET", "FCO_MET", "FCF_MET",
    "Divida_Liquida_MET", "PL_MET", "Divida_Bruta_MET",
    "AT_MET", "DVA_Despesas_Fin_MET",
    "PC_MET", "PNC_MET", "Outros_PC_MET",
    "LUB_MET",

    # ===== Fundamental – variações Q (quarter-over-quarter) =====
    "RL_Q_Change", "LL_Q_Change", "EBITDA_Q_Change",
    "Preco_Abertura_Q_Change", "Preco_Fechamento_Q_Change",
    "LPA_Q_Change", "ROA_Q_Change", "ROE_Q_Change", "MEB_Q_Change",
    "CRESC_RL_12M_Q_Change", "CRESC_LL_12M_Q_Change", "CRESC_EBITDA_12M_Q_Change",
    "CAPEX_Q_Change", "FCO_Q_Change", "FCF_Q_Change",
    "Divida_Liquida_Q_Change", "PL_Q_Change", "Divida_Bruta_Q_Change",
    "AT_Q_Change", "DVA_Despesas_Fin_Q_Change",
    "PC_Q_Change", "PNC_Q_Change", "Outros_PC_Q_Change",
    "LUB_Q_Change",

    # ===== Fundamental – variações Y (year-over-year) =====
    "RL_Y_Change", "LL_Y_Change", "EBITDA_Y_Change",
    "Preco_Abertura_Y_Change", "Preco_Fechamento_Y_Change",
    "LPA_Y_Change", "ROA_Y_Change", "ROE_Y_Change", "MEB_Y_Change",
    "CRESC_RL_12M_Y_Change", "CRESC_LL_12M_Y_Change", "CRESC_EBITDA_12M_Y_Change",
    "CAPEX_Y_Change", "FCO_Y_Change", "FCF_Y_Change",
    "Divida_Liquida_Y_Change", "PL_Y_Change", "Divida_Bruta_Y_Change",
    "AT_Y_Change", "DVA_Despesas_Fin_Y_Change",
    "PC_Y_Change", "PNC_Y_Change", "Outros_PC_Y_Change",
    "LUB_Y_Change",

    # ===== EPS Surprise Features =====
    "EPS_EarningsSurprise",
    "EPS_Earnings_Surprise_Backward_Diff",
    "EPS_Earnings_Surprise_Backward_Ave_Diff",

    # ===== Momentum & Technical Indicators =====
    "MA5", "MA50", "MA200",
    "RSI9", "RSI14", "RSI30",
    "MA5_50", "MA5_200", "MA50_200",
    "MOM_1M", "MOM_3M", "MOM_6M", "MOM_12M"
]


In [101]:
ID_COLS      = ["Ticker", "EventTradeDate", "AnnounceDate", "Date", "SectorName", "SectorID", "Quarter", "EstimationLen", "FundSource", "Data", "Empresa" ]
TARGET_COLS  = ["CAR_30D", "CAR_Sign"]   # pode incluir outros alvos se existirem, mas não entram em X
LEAK_PREFIX  = ("CAR_", "AR_", "RET_")  # colunas proibidas no espaço de features

# Split temporal (ajuste se precisar)
TRAIN_END_YEAR = 2016
VAL_END_YEAR   = 2016   # val = (TRAIN_END_YEAR+1) .. VAL_END_YEAR
# test = >= (VAL_END_YEAR+1)

In [102]:
# -------------------- Helpers --------------------


def parse_date_br_any(sr: pd.Series) -> pd.Series:
    """
    Converte a coluna de datas de publicação para datetime,
    assumindo SEMPRE padrão brasileiro (DD/MM/AAAA) quando houver ambiguidade.

    Regras:
    - Se estiver no padrão ISO 'YYYY-MM-DD', usamos isso direto (não é ambíguo).
    - Se estiver no padrão brasileiro 'DD/MM/YYYY', interpretamos como dia/mês/ano.
    - Se vier em qualquer outro formato, tentamos parse com dayfirst=True.
    - No final, retornamos datetime normalizado (sem hora).
    """

    s = sr.astype(str).str.strip()

    # 1) tenta ISO claro: 2024-03-31
    iso_mask = s.str.match(r"^\d{4}-\d{2}-\d{2}$")
    out_iso = pd.to_datetime(
        s.where(iso_mask),
        format="%Y-%m-%d",
        errors="coerce"
    )

    # 2) tenta BR claro: 31/03/2024
    br_mask = s.str.match(r"^\d{2}/\d{2}/\d{4}$")
    out_br = pd.to_datetime(
        s.where(br_mask),
        format="%d/%m/%Y",
        dayfirst=True,
        errors="coerce"
    )

    # 3) começa com ISO e preenche lacunas com BR
    out = out_iso.fillna(out_br)

    # 4) fallback genérico:
    #    qualquer coisa que sobrou a gente interpreta assumindo padrão brasileiro (dayfirst=True)
    still_nat = out.isna()
    if still_nat.any():
        out_fallback = pd.to_datetime(
            s[still_nat],
            errors="coerce",
            dayfirst=True   # <- força semântica brasileira
        )
        out.loc[still_nat] = out_fallback

    # 5) normaliza para "apenas a data" (zera hora)
    out = out.dt.normalize()

    return out

def to_datetime_br_inplace(df, cols):
    for c in cols:
        if c in df.columns:
            df[c] = parse_date_br_any(df[c])

def pick_feature_cols(df):
    # Candidatas: numéricas
    numeric_cols = df.select_dtypes(include=[np.number]).columns.tolist()
    # Remove targets
    candidate = [c for c in numeric_cols if c in FEATS]
    # Remove leaks por prefixo
    #candidate = [c for c in candidate if not c.startswith(LEAK_PREFIX)]
    return candidate

def zscore_per_firm(df, feat_cols, firm_col="Ticker"):
    """
    Etapa 2: padronização por empresa. Retorna df_znorm e 'firm_stats' com mean/std por empresa e feature.
    """
    firm_stats = {}
    df = df.copy()
    # calcula mean/std por empresa APENAS com histórico disponível (ordem temporal já garantida)
    g = df.groupby(firm_col)
    for firm, sub in g:
        stats_firm = {}
        for col in feat_cols:
            x = sub[col].astype(float).values
            mu = np.nanmean(x)
            sd = np.nanstd(x, ddof=0)
            # evita divisão por zero; se sd=0 → mantém como 1 para não explodir
            if not np.isfinite(sd) or sd == 0:
                sd = 1.0
            stats_firm[col] = {"mean": float(mu), "std": float(sd)}
            df.loc[sub.index, col] = (x - mu) / sd
        firm_stats[firm] = stats_firm
    return df, firm_stats

def winsorize_train_fit(train_df, feat_cols, lower=0.01, upper=0.99):
    """
    Etapa 3 (ajuste NO TREINO): calcula limites de winsorização por coluna (sobre treino já zscore por empresa).
    """
    bounds = {}
    for col in feat_cols:
        s = pd.to_numeric(train_df[col], errors="coerce")
        q_low, q_high = s.quantile([lower, upper])
        bounds[col] = {"q_low": float(q_low), "q_high": float(q_high)}
    return bounds

def winsorize_apply(df, feat_cols, bounds):
    df = df.copy()
    for col in feat_cols:
        lo = bounds[col]["q_low"]
        hi = bounds[col]["q_high"]
        df[col] = np.clip(pd.to_numeric(df[col], errors="coerce"), lo, hi)
    return df

def global_standard_fit(train_df, feat_cols):
    """
    Etapa 6 (ajuste global NO TREINO): média e DP globais do treino (após winsor).
    """
    stats = {}
    for col in feat_cols:
        x = pd.to_numeric(train_df[col], errors="coerce").values
        mu = np.nanmean(x)
        sd = np.nanstd(x, ddof=0)
        if not np.isfinite(sd) or sd == 0:
            sd = 1.0
        stats[col] = {"mean": float(mu), "std": float(sd)}
    return stats

def global_standard_apply(df, feat_cols, stats):
    df = df.copy()
    for col in feat_cols:
        mu = stats[col]["mean"]
        sd = stats[col]["std"]
        df[col] = (pd.to_numeric(df[col], errors="coerce") - mu) / sd
    return df

In [103]:
# -------------------- Load & basic checks --------------------
df = pd.read_csv(CSV_PATH, low_memory=False)
date_cols = [c for c in ["EventTradeDate","AnnounceDate","Date"] if c in df.columns]
to_datetime_br_inplace(df, date_cols)

# coluna de data de referência do evento
event_col = "EventTradeDate" if "EventTradeDate" in df.columns else (date_cols[0] if date_cols else None)
if event_col is None:
    raise ValueError("Não encontrei coluna de data do evento (ex.: 'EventTradeDate').")

# ordena por empresa e data (Etapa 1)
df = df.sort_values(["Ticker", event_col]).reset_index(drop=True)

# escolhe features
feat_cols = pick_feature_cols(df)
if not feat_cols:
    raise ValueError("Nenhuma feature numérica elegível encontrada. Verifique prefixos e TARGET_COLS.")

In [104]:
# -------------------- Split temporal --------------------
year = df[event_col].dt.year
mask_train = year <= TRAIN_END_YEAR
mask_val   = (year > TRAIN_END_YEAR) & (year <= VAL_END_YEAR)
mask_test  = year > VAL_END_YEAR

df_train = df.loc[mask_train].copy()
df_val   = df.loc[mask_val].copy()
df_test  = df.loc[mask_test].copy()

In [105]:
# -------------------- Etapa 2: Z-score por empresa (em TODOS os conjuntos)
# Importante: as estatísticas intraempresa são calculadas dentro de cada subset atual.
# (Para replicar rigidamente o paper, você pode optar por calcular no treino e aplicar nas datas posteriores
#  somente com as estatísticas até o ponto anterior; aqui salvamos o "snapshot" por subset.)

df_train_z, firm_stats_train = zscore_per_firm(df_train, feat_cols, firm_col="Ticker")
df_val_z,   firm_stats_val   = zscore_per_firm(df_val,   feat_cols, firm_col="Ticker")
df_test_z,  firm_stats_test  = zscore_per_firm(df_test,  feat_cols, firm_col="Ticker")

# Opcional: consolidar stats (mantendo separado por split para auditoria)
firm_stats = {
    "train": firm_stats_train,
    "val":   firm_stats_val,
    "test":  firm_stats_test
}
with open(os.path.join(OUT_DIR, "firm_stats.json"), "w") as f:
    json.dump(firm_stats, f, indent=2)

C:\Users\thgcn\AppData\Local\Temp\ipykernel_23432\4132829920.py:80: RuntimeWarning: Mean of empty slice
  mu = np.nanmean(x)
c:\Users\thgcn\anaconda3\envs\fin-ml\Lib\site-packages\numpy\lib\nanfunctions.py:1879: RuntimeWarning: Degrees of freedom <= 0 for slice.
  var = nanvar(a, axis=axis, dtype=dtype, out=out, ddof=ddof,
C:\Users\thgcn\AppData\Local\Temp\ipykernel_23432\4132829920.py:80: RuntimeWarning: Mean of empty slice
  mu = np.nanmean(x)
c:\Users\thgcn\anaconda3\envs\fin-ml\Lib\site-packages\numpy\lib\nanfunctions.py:1879: RuntimeWarning: Degrees of freedom <= 0 for slice.
  var = nanvar(a, axis=axis, dtype=dtype, out=out, ddof=ddof,
C:\Users\thgcn\AppData\Local\Temp\ipykernel_23432\4132829920.py:80: RuntimeWarning: Mean of empty slice
  mu = np.nanmean(x)
c:\Users\thgcn\anaconda3\envs\fin-ml\Lib\site-packages\numpy\lib\nanfunctions.py:1879: RuntimeWarning: Degrees of freedom <= 0 for slice.
  var = nanvar(a, axis=axis, dtype=dtype, out=out, ddof=ddof,
C:\Users\thgcn\AppData\Lo

In [95]:
# -------------------- Etapa 3: Winsorização ajustada no TREINO e aplicada em val/test
winsor_bounds = winsorize_train_fit(df_train_z, feat_cols, lower=0.01, upper=0.99)
with open(os.path.join(OUT_DIR, "winsor_bounds_train.json"), "w") as f:
    json.dump(winsor_bounds, f, indent=2)

df_train_w = winsorize_apply(df_train_z, feat_cols, winsor_bounds)
df_val_w   = winsorize_apply(df_val_z,   feat_cols, winsor_bounds)
df_test_w  = winsorize_apply(df_test_z,  feat_cols, winsor_bounds)

In [96]:
# -------------------- Etapa 6: Padronização global ajustada no TREINO e aplicada em val/test
global_stats = global_standard_fit(df_train_w, feat_cols)
with open(os.path.join(OUT_DIR, "global_stats_train.json"), "w") as f:
    json.dump(global_stats, f, indent=2)

df_train_final = global_standard_apply(df_train_w, feat_cols, global_stats)
df_val_final   = global_standard_apply(df_val_w,   feat_cols, global_stats)
df_test_final  = global_standard_apply(df_test_w,  feat_cols, global_stats)

In [99]:
# -------------------- Salvar CSVs finais --------------------
train_path = os.path.join(OUT_DIR, "train_processed.csv")
val_path   = os.path.join(OUT_DIR, "val_processed.csv")
test_path  = os.path.join(OUT_DIR, "test_processed.csv")

df_train_final.to_csv(train_path, index=False)
df_val_final.to_csv(val_path, index=False)
df_test_final.to_csv(test_path, index=False)

In [100]:
# -------------------- Relatório rápido --------------------
summary = []
summary.append(f"Linhas totais: {len(df):,}")
summary.append(f"Train: {len(df_train_final):,}  Val: {len(df_val_final):,}  Test: {len(df_test_final):,}")
summary.append(f"Nº de features usadas: {len(feat_cols)}")
summary.append("Primeiras 15 features:\n  - " + "\n  - ".join(feat_cols[:15]))
summary_txt = "\n".join(summary)

with open(os.path.join(OUT_DIR, "summary.txt"), "w", encoding="utf-8") as f:
    f.write(summary_txt)

print(summary_txt)
print("\nArquivos gerados:")
print(" -", train_path)
print(" -", val_path)
print(" -", test_path)
print(" -", os.path.join(OUT_DIR, "firm_stats.json"))
print(" -", os.path.join(OUT_DIR, "winsor_bounds_train.json"))
print(" -", os.path.join(OUT_DIR, "global_stats_train.json"))
print(" -", os.path.join(OUT_DIR, "summary.txt"))

Linhas totais: 4,919
Train: 2,881  Val: 499  Test: 1,539
Nº de features usadas: 88
Primeiras 15 features:
  - RL_MET
  - RL_Q_Change
  - RL_Y_Change
  - LL_MET
  - LL_Q_Change
  - LL_Y_Change
  - EBITDA_MET
  - EBITDA_Q_Change
  - EBITDA_Y_Change
  - Preco_Abertura_MET
  - Preco_Abertura_Q_Change
  - Preco_Abertura_Y_Change
  - Preco_Fechamento_MET
  - Preco_Fechamento_Q_Change
  - Preco_Fechamento_Y_Change

Arquivos gerados:
 - pead_preproc\train_processed.csv
 - pead_preproc\val_processed.csv
 - pead_preproc\test_processed.csv
 - pead_preproc\firm_stats.json
 - pead_preproc\winsor_bounds_train.json
 - pead_preproc\global_stats_train.json
 - pead_preproc\summary.txt
